In [1]:
import pandas as pd
import numpy as np

## 时间序列

### 时间点、时间段、时间间隔

In [2]:
now = pd.Timestamp.now()
now_shanghai = now.tz_localize("Asia/Shanghai")
now_tokyo = now_shanghai.tz_convert("Asia/Tokyo")
print(("本地时间:", now))
print(("上海时区:", now_shanghai))
print(("东京时区:", now_tokyo))

('本地时间:', Timestamp('2022-07-04 14:39:06.784710'))
('上海时区:', Timestamp('2022-07-04 14:39:06.784710+0800', tz='Asia/Shanghai'))
('东京时区:', Timestamp('2022-07-04 15:39:06.784710+0900', tz='Asia/Tokyo'))


In [3]:
now_shanghai == now_tokyo

True

In [5]:
#获取常用表示时区的字符串
import pytz
%omit pytz.common_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
...


In [14]:
now_day = pd.Period.now(freq="D")
now_hour = pd.Period.now(freq="H")
%C now_day; now_hour

         now_day                       now_hour           
-------------------------  -------------------------------
Period('2022-07-04', 'D')  Period('2022-07-04 15:00', 'H')


In [11]:
#代码不兼容，放弃
from pandas.tseries import frequencies
list(frequencies._period_code_map.keys())
frequencies._period_alias_dictionary()

TypeError: 'str' object is not callable

In [12]:
#周时间段
now_week_sun = pd.Period.now(freq="W")
now_week_mon = pd.Period.now(freq="W-MON")
%C now_week_sun; now_week_mon

              now_week_sun                              now_week_mon              
----------------------------------------  ----------------------------------------
Period('2022-07-04/2022-07-10', 'W-SUN')  Period('2022-06-28/2022-07-04', 'W-MON')


In [15]:
%C now_day.start_time; now_day.end_time

       now_day.start_time                      now_day.end_time             
--------------------------------  ------------------------------------------
Timestamp('2022-07-04 00:00:00')  Timestamp('2022-07-04 23:59:59.999999999')


In [19]:
# now_shanghai.to_period("H")
now.to_period("H")

Period('2022-07-04 14:00', 'H')

In [20]:
%C now.year; now.month; now.day; now.dayofweek; now.dayofyear; now.hour

now.year  now.month  now.day  now.dayofweek  now.dayofyear  now.hour
--------  ---------  -------  -------------  -------------  --------
2022      7          4        0              185            14      


In [21]:
national_day = pd.Timestamp("2022-10-1")
td = national_day - pd.Timestamp.now()
td

Timedelta('88 days 08:43:01.099391')

In [22]:
national_day + pd.Timedelta("20 days 10:20:30") 

Timestamp('2022-10-21 10:20:30')

In [23]:
%C td.days; td.seconds; td.microseconds

td.days  td.seconds  td.microseconds
-------  ----------  ---------------
88       31381       99391          


In [24]:
print((pd.Timedelta(days=10, hours=1, minutes=2, seconds=10.5)))
print((pd.Timedelta(seconds=100000)))

10 days 01:02:10.500000
1 days 03:46:40


### 时间序列

In [36]:
def random_timestamps(start, end, freq, count):
    index = pd.date_range(start, end, freq=freq) #生成时间序列
    locations = np.random.choice(np.arange(len(index)), size=count, replace=False) #随机抽取整数用于索引下标
    locations.sort() #排序
    return index[locations]

np.random.seed(42)
ts_index = random_timestamps("2015-01-01", "2015-10-01", freq="Min", count=5)
pd_index = ts_index.to_period("M")
td_index = pd.TimedeltaIndex(np.diff(ts_index)) #时间想减转成timedelta

print(ts_index, sep="\n")
print(pd_index, sep="\n")
print(td_index, sep="\n")

DatetimeIndex(['2015-01-15 16:12:00', '2015-02-15 08:04:00',
               '2015-02-28 12:30:00', '2015-08-06 02:40:00',
               '2015-08-18 13:13:00'],
              dtype='datetime64[ns]', freq=None)
PeriodIndex(['2015-01', '2015-02', '2015-02', '2015-08', '2015-08'], dtype='period[M]')
TimedeltaIndex(['30 days 15:52:00', '13 days 04:26:00', '158 days 14:10:00',
                '12 days 10:33:00'],
               dtype='timedelta64[ns]', freq=None)


In [37]:
%C ts_index.dtype; pd_index.dtype; td_index.dtype

 ts_index.dtype   pd_index.dtype   td_index.dtype 
----------------  --------------  ----------------
dtype('<M8[ns]')  period[M]       dtype('<m8[ns]')


In [38]:
%C ts_index.weekday; pd_index.month; td_index.seconds

             ts_index.weekday                             pd_index.month                                    td_index.seconds                   
------------------------------------------  ------------------------------------------  -------------------------------------------------------
Int64Index([3, 6, 5, 3, 1], dtype='int64')  Int64Index([1, 2, 2, 8, 8], dtype='int64')  Int64Index([57120, 15960, 51000, 37980], dtype='int64')


In [39]:
#按周期移动时间
ts_index.shift(1, "H")

DatetimeIndex(['2015-01-15 17:12:00', '2015-02-15 09:04:00',
               '2015-02-28 13:30:00', '2015-08-06 03:40:00',
               '2015-08-18 14:13:00'],
              dtype='datetime64[ns]', freq=None)

In [21]:
#月份不是精确的时间单位，所以时间会移动到该月末尾。因为一个月天数不固定，年也是同样的道理。
ts_index.shift(1, "M")

DatetimeIndex(['2015-01-31 16:12:00', '2015-02-28 08:04:00',
               '2015-03-31 12:30:00', '2015-08-31 02:40:00',
               '2015-08-31 13:13:00'],
              dtype='datetime64[ns]', freq=None, tz=None)

In [41]:
#按日期取整
ts_index.normalize()

DatetimeIndex(['2015-01-15', '2015-02-15', '2015-02-28', '2015-08-06',
               '2015-08-18'],
              dtype='datetime64[ns]', freq=None)

In [43]:
#使用周期来取整
ts_index.to_period("H").to_timestamp()

DatetimeIndex(['2015-01-15 16:00:00', '2015-02-15 08:00:00',
               '2015-02-28 12:00:00', '2015-08-06 02:00:00',
               '2015-08-18 13:00:00'],
              dtype='datetime64[ns]', freq=None)

In [44]:
#生成以时间序列为index的系列
ts_series = pd.Series(list(range(5)), index=ts_index)
ts_series

2015-01-15 16:12:00    0
2015-02-15 08:04:00    1
2015-02-28 12:30:00    2
2015-08-06 02:40:00    3
2015-08-18 13:13:00    4
dtype: int64

In [45]:
#时间index筛选
ts_series.between_time("9:00", "18:00")

2015-01-15 16:12:00    0
2015-02-28 12:30:00    2
2015-08-18 13:13:00    4
dtype: int64

In [47]:
#操作时间index
ts_series.shift(1, freq="D")

2015-01-16 16:12:00    0
2015-02-16 08:04:00    1
2015-03-01 12:30:00    2
2015-08-07 02:40:00    3
2015-08-19 13:13:00    4
dtype: int64

In [50]:
#tshift函数升级为shift
pd_series = pd.Series(range(5), index=pd_index)
td_series = pd.Series(range(4), index=td_index)
%C pd_series.shift(1,freq="M"); td_series.shift(10, freq="H")

pd_series.shift(1,freq="M")  td_series.shift(10, freq="H")
---------------------------  -----------------------------
2015-02    0                 31 days 01:52:00     0       
2015-03    1                 13 days 14:26:00     1       
2015-03    2                 159 days 00:10:00    2       
2015-09    3                 12 days 20:33:00     3       
2015-09    4                 dtype: int64                 
Freq: M, dtype: int64                                     


In [51]:
#将时间序列作为列数据
ts_data = pd.Series(ts_index)
pd_data = pd.Series(pd_index)
td_data = pd.Series(td_index)
%C ts_data.dtype; pd_data.dtype; td_data.dtype

 ts_data.dtype    pd_data.dtype   td_data.dtype  
----------------  -------------  ----------------
dtype('<M8[ns]')  period[M]      dtype('<m8[ns]')


In [52]:
#时间序列作为数据时可以通过dt属性调用时间属性
%C ts_data.dt.hour; pd_data.dt.month; td_data.dt.days

ts_data.dt.hour  pd_data.dt.month  td_data.dt.days
---------------  ----------------  ---------------
0    16          0    1            0     30       
1     8          1    2            1     13       
2    12          2    2            2    158       
3     2          3    8            3     12       
4    13          4    8            dtype: int64   
dtype: int64     dtype: int64                     
